# Tests

In [1]:
import os
import sys

#dir = os.path.abspath(__file__)
#dir    = os.getcwd()
#pkgdir = os.path.dirname(os.path.dirname(dir)

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from lib.nanuq import Nanuq
